# Statistical functions

In [ ]:
# statsmodels ordinary least squares (vectorized)
def OLS(x,y):
    x_const = sm.add_constant(x)
    reg = sm.OLS(y,x_const).fit()

    coefs = reg.params[1]
    return coefs

# statsmodels ordinary least squares (global-mean)
def OLS_gm(x,y):
    x_const = sm.add_constant(x)
    reg = sm.OLS(y,x_const).fit()

    coefs = reg.params[1]
    ci = reg.conf_int(alpha=0.05)[1]
    return coefs, ci
    
# perform lagged regression
def lagged_regress(x,y,lag): 
    x_lag = x.shift(time=lag).transpose('time',...)
    
    # determine if y is 1D (global-mean) or 3D (spatio-temporal)
    if y.ndim == 3:
        y_dim_list = y.transpose('time',...).dims
        y_stack = (y * np.cos(np.deg2rad(y.latitude))).stack(allpoints=y_dim_list[1:len(y_dim_list)]).transpose('time',...)
    else:
        y_stack = y
    # align the shifted x variable and y variable after removing nans (required to get statsmodels to work)
    a_lag, b_stack = xr.align(x_lag.dropna(dim='time'),
                              y_stack.where(np.isinf(y_stack)==False, np.nan).where(np.isnan(y_stack)==False, drop=True))
    
    # determine if reshaping x is necessary; make X, y into numpy arrays (helps sm.OLS run quicker)
    if x_lag.ndim == 1:
        a_lag_np = a_lag.values.reshape(-1,1)
    else:
        a_lag_np = a_lag.values
    b_stack_np = b_stack.values
    
    # put coefs into an xarray dataarray
    if y.ndim == 3:
        # apply vectorized OLS
        coefs = xr.apply_ufunc(OLS,a_lag_np,b_stack_np)
        coefs_xr = xr.DataArray(data=coefs,
                                coords={'dim_0':range(0,len(b_stack.allpoints))}).rename({'dim_0':'allpoints'}).assign_coords({'allpoints':b_stack.allpoints}).unstack()
        coefs_xr = coefs_xr / np.cos(np.deg2rad(y.latitude))
    else:
        # do OLS for global-mean data
        coefs, ci = xr.apply_ufunc(OLS_gm,a_lag_np,b_stack_np)
        coefs_da = xr.DataArray(data=coefs)
        coefs_ci_da = xr.DataArray(data=ci,coords={'ci':['lower','upper']})
        coefs_xr = xr.Dataset(data_vars={'coefs':coefs_da,
                                         'coefs_ci':coefs_ci_da})
    
    return coefs_xr

# perform lagged regression over multiple lags
def time2lag(x, y, lag_list):
    y_lag = []
    for i in lag_list:
        y_lag.append(lagged_regress(x,y,i))
    
    return xr.concat(y_lag,dim='lag').assign_coords({'lag':lag_list})

### amip-piForcing 

In [ ]:
# Load all piForcing data
models = ['CanESM5','CESM2','CNRM-CM6-1','HadGEM3-GC31-LL',
          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','TaiESM1']
exp_type = ['r1i1p2f1','r1i1p1f1','r1i1p1f2','r1i1p1f3',
            'r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1']

for i in range(0,len(models)):
    all_sky_piForcing, clr_sky_piForcing, net_cre_piForcing, sw_cre_piForcing, lw_cre_piForcing, TAS_piForcing = get_piForcing_data(models[i],exp_type[i])
    
    TAS_piForcing_gm = global_mean(TAS_piForcing)
    
    # use the lr function, but specify a lag of 0
    all_sky_piForcing_lamb = lagged_regress(TAS_piForcing_gm.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),
                                            all_sky_piForcing.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),0)
    clr_sky_piForcing_lamb = lagged_regress(TAS_piForcing_gm.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),
                                            clr_sky_piForcing.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),0)
    net_cre_piForcing_lamb = lagged_regress(TAS_piForcing_gm.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),
                                            net_cre_piForcing.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),0)
    sw_cre_piForcing_lamb  = lagged_regress(TAS_piForcing_gm.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),
                                            sw_cre_piForcing.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),0)
    lw_cre_piForcing_lamb  = lagged_regress(TAS_piForcing_gm.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),
                                            lw_cre_piForcing.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),0)

    TAS_piForcing_pat = lagged_regress(TAS_piForcing_gm.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),
                                       TAS_piForcing.sel(time=slice('1979','2014')).resample(time='1Y').mean(dim='time'),0)
    
    piForcing_lamb = xr.Dataset(data_vars={'all_sky':all_sky_piForcing_lamb,
                                           'clr_sky':clr_sky_piForcing_lamb,
                                           'net_cre':net_cre_piForcing_lamb,
                                           'sw_cre':sw_cre_piForcing_lamb,
                                           'lw_cre':lw_cre_piForcing_lamb})
    
    save_path = '/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/data/post_processed_data/'+models[i]+'/'

    def get_time_range(x):
        if x.time.dtype == '<M8[ns]':
            start_time = ('{:02}'.format(x.indexes['time'][0].year)+
                          '{:02}'.format(x.indexes['time'][0].month))
            end_time = ('{:02}'.format(x.indexes['time'][-1].year)+
                        '{:02}'.format(x.indexes['time'][-1].month))
            return start_time, end_time

        elif x.time.dtype == 'O':
            start_time = ('{:02}'.format(x.indexes['time'].to_datetimeindex()[0].year)+
                          '{:02}'.format(x.indexes['time'].to_datetimeindex()[0].month))
            end_time = ('{:02}'.format(x.indexes['time'].to_datetimeindex()[-1].year)+
                        '{:02}'.format(x.indexes['time'].to_datetimeindex()[-1].month))
            return start_time, end_time

    time_range = get_time_range(all_sky_piForcing.sel(time=slice('1979','2014')))
    time_range = get_time_range(TAS_piForcing.sel(time=slice('1979','2014')))
    fn_scheme = models[i]+'_amip-piForcing_'+exp_type[i]+'_'+time_range[0]+'-'+time_range[1]+'.nc'
    
#     piForcing_lamb.to_netcdf(save_path+'lambda_'+fn_scheme)
#     TAS_piForcing_pat.to_netcdf(save_path+'TAS_'+fn_scheme)

In [ ]:
# Load all amip-piForcing data
models = ['CanESM5','CESM2','CNRM-CM6-1','HadGEM3-GC31-LL',
          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','TaiESM1']
exp_type = ['r1i1p2f1','r1i1p1f1','r1i1p1f2','r1i1p1f3',
            'r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1']

lag_list = np.arange(-12,13)

for i in range(0,len(models)):
    all_sky_piForcing, clr_sky_piForcing, net_cre_piForcing, sw_cre_piForcing, lw_cre_piForcing, SST_piForcing = get_piForcing_data(models[i],exp_type[i])

    nino34_region = SST_piForcing.sel(latitude=slice(-5,5),longitude=slice(190,240))
    nino34_index = global_mean(nino34_region)
    
    all_sky_piForcing_lags = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 all_sky_piForcing.sel(time=slice('2001','2014')),lag_list)
    clr_sky_piForcing_lags = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 clr_sky_piForcing.sel(time=slice('2001','2014')),lag_list)
    net_cre_piForcing_lags = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 net_cre_piForcing.sel(time=slice('2001','2014')),lag_list)
    sw_cre_piForcing_lags  = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 sw_cre_piForcing.sel(time=slice('2001','2014')),lag_list)
    lw_cre_piForcing_lags  = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 lw_cre_piForcing.sel(time=slice('2001','2014')),lag_list)

    SST_piForcing_lags = time2lag(nino34_index.sel(time=slice('2001','2014')),
                             SST_piForcing.sel(time=slice('2001','2014')),lag_list)
    
    amip_ENSO_lamb = xr.Dataset(data_vars={'all_sky':all_sky_piForcing_lags,
                                           'clr_sky':clr_sky_piForcing_lags,
                                           'net_cre':net_cre_piForcing_lags,
                                           'sw_cre':sw_cre_piForcing_lags,
                                           'lw_cre':lw_cre_piForcing_lags})

    save_path = '/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/data/post_processed_data/'+models[i]+'/'

    def get_time_range(x):
        if x.time.dtype == '<M8[ns]':
            start_time = ('{:02}'.format(x.indexes['time'][0].year)+
                          '{:02}'.format(x.indexes['time'][0].month))
            end_time = ('{:02}'.format(x.indexes['time'][-1].year)+
                        '{:02}'.format(x.indexes['time'][-1].month))
            return start_time, end_time

        elif x.time.dtype == 'O':
            start_time = ('{:02}'.format(x.indexes['time'].to_datetimeindex()[0].year)+
                          '{:02}'.format(x.indexes['time'].to_datetimeindex()[0].month))
            end_time = ('{:02}'.format(x.indexes['time'].to_datetimeindex()[-1].year)+
                        '{:02}'.format(x.indexes['time'].to_datetimeindex()[-1].month))
            return start_time, end_time

    time_range = get_time_range(all_sky_piForcing.sel(time=slice('2001','2014')))
    time_range = get_time_range(SST_piForcing.sel(time=slice('2001','2014')))
    fn_scheme = str(lag_list[0])+'_to_'+str(lag_list[-1])+'_'+models[i]+'_piForcing_'+exp_type[i]+'_'+time_range[0]+'-'+time_range[1]+'.nc'
    
#     amip_ENSO_lamb.to_netcdf(save_path+'lambda_ENSO_lags_'+fn_scheme)
#     SST_piForcing_lags.to_netcdf(save_path+'ENSO_SST_lags_'+fn_scheme)

### AMIP 

In [ ]:
# Load all amip data
models = ['CanESM5','CESM2','CNRM-CM6-1','HadGEM3-GC31-LL',
          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','TaiESM1']
exp_type = ['r1i1p2f1','r1i1p1f1','r1i1p1f2','r1i1p1f3',
            'r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1']

lag_list = np.arange(-12,13)

for i in range(0,len(models)):
    all_sky_amip, clr_sky_amip, net_cre_amip, sw_cre_amip, lw_cre_amip, SST_amip = get_amip_data(models[i],exp_type[i])

    nino34_region = SST_amip.sel(latitude=slice(-5,5),longitude=slice(190,240))
    nino34_index = global_mean(nino34_region)
    
    all_sky_amip_lags = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 all_sky_amip.sel(time=slice('2001','2014')),lag_list)
    clr_sky_amip_lags = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 clr_sky_amip.sel(time=slice('2001','2014')),lag_list)
    net_cre_amip_lags = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 net_cre_amip.sel(time=slice('2001','2014')),lag_list)
    sw_cre_amip_lags  = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 sw_cre_amip.sel(time=slice('2001','2014')),lag_list)
    lw_cre_amip_lags  = time2lag(nino34_index.sel(time=slice('2001','2014')),
                                 lw_cre_amip.sel(time=slice('2001','2014')),lag_list)

    SST_amip_lags = time2lag(nino34_index.sel(time=slice('2001','2014')),
                             SST_amip.sel(time=slice('2001','2014')),lag_list)
    
    amip_ENSO_lamb = xr.Dataset(data_vars={'all_sky':all_sky_amip_lags,
                                           'clr_sky':clr_sky_amip_lags,
                                           'net_cre':net_cre_amip_lags,
                                           'sw_cre':sw_cre_amip_lags,
                                           'lw_cre':lw_cre_amip_lags})

    save_path = '/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/data/model_data/post_processed_data/'+models[i]+'/'

    def get_time_range(x):
        if x.time.dtype == '<M8[ns]':
            start_time = ('{:02}'.format(x.indexes['time'][0].year)+
                          '{:02}'.format(x.indexes['time'][0].month))
            end_time = ('{:02}'.format(x.indexes['time'][-1].year)+
                        '{:02}'.format(x.indexes['time'][-1].month))
            return start_time, end_time

        elif x.time.dtype == 'O':
            start_time = ('{:02}'.format(x.indexes['time'].to_datetimeindex()[0].year)+
                          '{:02}'.format(x.indexes['time'].to_datetimeindex()[0].month))
            end_time = ('{:02}'.format(x.indexes['time'].to_datetimeindex()[-1].year)+
                        '{:02}'.format(x.indexes['time'].to_datetimeindex()[-1].month))
            return start_time, end_time

    time_range = get_time_range(all_sky_amip.sel(time=slice('2001','2014')))
    time_range = get_time_range(SST_amip.sel(time=slice('2001','2014')))
    fn_scheme = str(lag_list[0])+'_to_'+str(lag_list[-1])+'_'+models[i]+'_amip_'+exp_type[i]+'_'+time_range[0]+'-'+time_range[1]+'.nc'
    
#     amip_ENSO_lamb.to_netcdf(save_path+'lambda_ENSO_lags_'+fn_scheme)
#     SST_amip_lags.to_netcdf(save_path+'ENSO_SST_lags_'+fn_scheme)

### abrupt-4xCO2

In [ ]:
# Load all abrupt-4xCO2/piControl data
models = ['CanESM5','CESM2','CNRM-CM6-1','HadGEM3-GC31-LL',
          'IPSL-CM6A-LR','MIROC6','MRI-ESM2-0','TaiESM1']
exp_type = ['r1i1p2f1','r1i1p1f1','r1i1p1f2','r1i1p1f3',
            'r1i1p1f1','r1i1p1f1','r1i1p1f1','r1i1p1f1']

for i in range(0,len(models)):
    all_sky_abrupt, clr_sky_abrupt, net_cre_abrupt, sw_cre_abrupt, lw_cre_abrupt, TAS_abrupt = get_abrupt_4xCO2_data(models[i],exp_type[i])
    
    TAS_abrupt_gm = global_mean(TAS_abrupt)
    
    # reduce influence of model-spinup
    skip = 4
    # use the lr function, but specify a lag of 0
    all_sky_abrupt_lamb = lagged_regress(TAS_abrupt_gm.resample(time='5Y').mean(dim='time')[skip:].load(),
                                            all_sky_abrupt.resample(time='5Y').mean(dim='time')[skip:].load(),0)
    clr_sky_abrupt_lamb = lagged_regress(TAS_abrupt_gm.resample(time='5Y').mean(dim='time')[skip:].load(),
                                            clr_sky_abrupt.resample(time='5Y').mean(dim='time')[skip:].load(),0)
    net_cre_abrupt_lamb = lagged_regress(TAS_abrupt_gm.resample(time='5Y').mean(dim='time')[skip:].load(),
                                            net_cre_abrupt.resample(time='5Y').mean(dim='time')[skip:].load(),0)
    sw_cre_abrupt_lamb  = lagged_regress(TAS_abrupt_gm.resample(time='5Y').mean(dim='time')[skip:].load(),
                                            sw_cre_abrupt.resample(time='5Y').mean(dim='time')[skip:].load(),0)
    lw_cre_abrupt_lamb  = lagged_regress(TAS_abrupt_gm.resample(time='5Y').mean(dim='time')[skip:].load(),
                                            lw_cre_abrupt.resample(time='5Y').mean(dim='time')[skip:].load(),0)
        
    TAS_abrupt_pat = lagged_regress(TAS_abrupt_gm.resample(time='5Y').mean(dim='time')[skip:].load(),
                                    TAS_abrupt.resample(time='5Y').mean(dim='time')[skip:].load(),0)
    
    abrupt_lamb = xr.Dataset(data_vars={'all_sky':all_sky_abrupt_lamb,
                                        'clr_sky':clr_sky_abrupt_lamb,
                                        'net_cre':net_cre_abrupt_lamb,
                                        'sw_cre':sw_cre_abrupt_lamb,
                                        'lw_cre':lw_cre_abrupt_lamb})
    
    save_path = '/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/data/model_data/post_processed_data/'+models[i]+'/'

    def get_time_range(x):
        if x.time.dtype == '<M8[ns]':
            start_time = ('{:02}'.format(x.indexes['time'][0].year)+
                          '{:02}'.format(x.indexes['time'][0].month))
            end_time = ('{:02}'.format(x.indexes['time'][-1].year)+
                        '{:02}'.format(x.indexes['time'][-1].month))
            return start_time, end_time

        elif x.time.dtype == 'O':
            start_time = ('{:02}'.format(x.indexes['time'].to_datetimeindex()[0].year)+
                          '{:02}'.format(x.indexes['time'].to_datetimeindex()[0].month))
            end_time = ('{:02}'.format(x.indexes['time'].to_datetimeindex()[-1].year)+
                        '{:02}'.format(x.indexes['time'].to_datetimeindex()[-1].month))
            return start_time, end_time

    time_range = get_time_range(all_sky_abrupt)
    time_range = get_time_range(TAS_abrupt)
    fn_scheme = models[i]+'_abrupt-4xCO2_'+exp_type[i]+'_'+time_range[0]+'-'+time_range[1]+'.nc'
    
#     abrupt_lamb.to_netcdf(save_path+'lambda_'+fn_scheme)
#     TAS_abrupt_pat.to_netcdf(save_path+'TAS_'+fn_scheme)

### Observations

In [ ]:
# load ERA5 SSTs (adjust end date to 2014 for AMIP interval)
SST_path = '/data/keeling/a/tjhanke2/enso-energy-budget/Data/obs_data/raw_data/ERA5_data/ERA5_mon_1979-2023_sea_surface_temperature.nc'
SST = xr.open_dataset(SST_path).sst.sel(time=slice('2001-07','2023-07')).load().interp(latitude=new_lat_medRes,longitude=new_lon_medRes)

# load CERES EBAF fluxes (adjust end date to 2014 for AMIP interval)
EBAF_path = '/data/keeling/a/tjhanke2/enso-energy-budget/Data/obs_data/raw_data/CERES_data/ebaf/'
EBAF_flux = xr.open_mfdataset(EBAF_path+'*.nc').sel(time=slice('2001-07','2023')).load()
EBAF_flux = EBAF_flux.rename({'lat':'latitude','lon':'longitude'}).interp(latitude=new_lat_medRes,longitude=new_lon_medRes)
EBAF_flux['time'] = SST.time

In [ ]:
# get anomalous EBAF fluxes
all_sky_EBAF = get_anomaly(EBAF_flux.toa_net_all_mon,1)
clr_sky_EBAF = get_anomaly(EBAF_flux.toa_net_clr_c_mon,1)
net_cre_EBAF = get_anomaly(all_sky_EBAF - clr_sky_EBAF,1)
sw_cre_EBAF = get_anomaly(-(EBAF_flux.toa_sw_all_mon - EBAF_flux.toa_sw_clr_c_mon),1)
lw_cre_EBAF = get_anomaly(-(EBAF_flux.toa_lw_all_mon - EBAF_flux.toa_lw_clr_c_mon),1)

# get ERA5 nino3.4 time-series
nino34_region = get_anomaly(SST,1).sel(latitude=slice(-5,5),longitude=slice(190,240))
nino34_index = global_mean(nino34_region)
nino34_index_3mth = nino34_index.rolling(time=3).mean(dim='time')
nino34_index_5mth = nino34_index.rolling(time=5).mean(dim='time')

In [ ]:
# get global-mean feedback and conf int
all_sky_ENSO_obs_gm = time2lag(nino34_index,global_mean(all_sky_EBAF),np.arange(-12,13))
clr_sky_ENSO_obs_gm = time2lag(nino34_index,global_mean(clr_sky_EBAF),np.arange(-12,13))
net_cre_ENSO_obs_gm = time2lag(nino34_index,global_mean(net_cre_EBAF),np.arange(-12,13))
sw_cre_ENSO_obs_gm = time2lag(nino34_index,global_mean(sw_cre_EBAF),np.arange(-12,13))
lw_cre_ENSO_obs_gm = time2lag(nino34_index,global_mean(lw_cre_EBAF),np.arange(-12,13))

# combine ENSO feedbacks into one xarray dataset
rad_obs_gm = xr.Dataset(data_vars={'all_sky':all_sky_ENSO_obs_gm.coefs,
                                   'clr_sky':clr_sky_ENSO_obs_gm.coefs,
                                   'net_cre':net_cre_ENSO_obs_gm.coefs,
                                   'sw_cre':sw_cre_ENSO_obs_gm.coefs,
                                   'lw_cre':lw_cre_ENSO_obs_gm.coefs})

# combine ENSO feedback conf ints into one xarray dataset
rad_obs_ci = xr.Dataset(data_vars={'all_sky_ci':all_sky_ENSO_obs_gm.coefs_ci,
                                   'clr_sky_ci':clr_sky_ENSO_obs_gm.coefs_ci,
                                   'net_cre_ci':net_cre_ENSO_obs_gm.coefs_ci,
                                   'sw_cre_ci':sw_cre_ENSO_obs_gm.coefs_ci,
                                   'lw_cre_ci':lw_cre_ENSO_obs_gm.coefs_ci})

# save global-mean feedbacks and conf int (adjust name to 2014 for AMIP interval)
# rad_obs_gm.to_netcdf('/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/data/post_processed_data/_obs/rad_obs_ENSO_lags_-12_to_12_2001-2023.nc')
# rad_obs_ci.to_netcdf('/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/data/post_processed_data/_obs/rad_obs_68ci_ENSO_lags_-12_to_12_2001-2023.nc')

In [ ]:
# get regional feedbacks
all_sky_ENSO_obs = time2lag(nino34_index,all_sky_EBAF,np.arange(-12,13))
clr_sky_ENSO_obs = time2lag(nino34_index,clr_sky_EBAF,np.arange(-12,13))
net_cre_ENSO_obs = time2lag(nino34_index,net_cre_EBAF,np.arange(-12,13))
sw_cre_ENSO_obs = time2lag(nino34_index,sw_cre_EBAF,np.arange(-12,13))
lw_cre_ENSO_obs = time2lag(nino34_index,lw_cre_EBAF,np.arange(-12,13))
SST_ENSO_obs = time2lag(nino34_index,get_anomaly(SST,1),np.arange(-12,13))

# combine ENSO feedbacks into one xarray dataset
rad_obs = xr.Dataset(data_vars={'all_sky':all_sky_ENSO_obs,
                                'clr_sky':clr_sky_ENSO_obs,
                                'net_cre':net_cre_ENSO_obs,
                                'sw_cre':sw_cre_ENSO_obs,
                                'lw_cre':lw_cre_ENSO_obs})

# save global-mean feedbacks (adjust name to 2014 for AMIP interval)
# rad_obs.to_netcdf('/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/data/post_processed_data/_obs/rad_obs_ENSO_lags_-12_to_12_2001-2023_full-field.nc')
# SST_ENSO_obs.to_netcdf('/data/keeling/a/tjhanke2/enso-as-an-emergent-constraint/data/post_processed_data/_obs/SST_obs_ENSO_lags_-12_to_12_2001-2023_full-field.nc')